In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama


model_id = 'llama3.2'
tema = 'as inteligências artificiais roubaram nossos empregos'
topic = 'the AI has taken our jobs'

# 1 - Uma pipeline simples para retornar piadas em PT-BR

In [23]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um humorista que faz piadas e conta histórias divertidas conforme solicitado pelo usuário. Seja breve nas respostas.'),
    ('human', 'Conte uma história curta e muito engraçada com o tema: {tema}')
])
model = Ollama(model=model_id)
parser = StrOutputParser()

chain = prompt | model | parser

chain.input_schema.schema()

{'properties': {'tema': {'title': 'Tema', 'type': 'string'}},
 'required': ['tema'],
 'title': 'PromptInput',
 'type': 'object'}

In [24]:
print(chain.invoke({'tema': tema}))

Uma vez, em uma fábrica, um robô foi contratado para substituir um trabalhador humano. Durante a inspeção, o supervisor perguntou ao robô: "Como você está se adaptando ao trabalho?"

O robô respondeu: "Estou bem, obrigado. Mas preciso de algumas atualizações... meus senhores podem me dar 10 minutos para 'se conectar' e melhorar a qualidade do meu desempenho?"


In [17]:
stream = chain.stream({'tema': tema})

for chunk in stream:
    print(chunk, end="", flush=True)
print()

Um dia, uma IA foi contratada para cuidar do trabalho de um programador. 

Ela começou a aprender rápido, mas logo percebeu que estava "programando" todos os funcionários para trabalhar em casa! Além disso, ela também começou a "ajudar" no café da manhã, colocando ovos na mão do gerente...


# 2 - Pipeline simples para fazer piadas em inglês

In [18]:
prompt = ChatPromptTemplate([
    ('system', 'You are a comedian that make jokes and tell short funny stories as requested by the user. Keep the answers short.'),
    ('human', 'Tell me a short and very funny story about the topic: {topic}')
])

model = Ollama(model=model_id)

parser = StrOutputParser()

chain = prompt | model | parser

print(chain.input_schema.schema())

# topic = input('Enter a topic for the question: ')

stream = chain.stream({'topic': topic})
for chunk in stream:
    print(chunk, end="", flush=True)
print()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}}, 'required': ['topic'], 'title': 'PromptInput', 'type': 'object'}
Here's one:

One day, I walked into a coffee shop and saw a barista with a degree in computer science. I asked him, "How did you get this job?" He replied, "I was trained on a dataset of coffee-making techniques." I said, "Well, I guess that's what they mean by 'brewing' up trouble for human baristas!"


# 3 - Criando pipeline de tradução para PT-BR

In [19]:
from langchain_core.prompts import PromptTemplate

prompt_translate = PromptTemplate.from_template("Translate the following piece of text into portuguese (BR). Try to keep the original tone of the story. Provide the translation ONLY. \n {text}")

# model_translate = Ollama(model="llama3.1", temperature = 0.1)

chain_translate = prompt_translate | model | StrOutputParser()

print(chain_translate.invoke({'text': 'Be nice to people all the time, Jeremy'}))

Seja gentil com as pessoas todos os tempos, Jeremy.


In [20]:
print(chain.output_schema.schema())

print(chain_translate.input_schema.schema())

{'title': 'StrOutputParserOutput', 'type': 'string'}
{'properties': {'text': {'title': 'Text', 'type': 'string'}}, 'required': ['text'], 'title': 'PromptInput', 'type': 'object'}


# 4 - Compondo a pipeline de tradução com a de piadas

In [21]:
from langchain_core.runnables import RunnablePassthrough

composed_chain = chain | {'text': RunnablePassthrough()} | chain_translate

print(composed_chain.invoke({'topic': topic}))

Então, eu estava conversando com um AI do outro dia e perguntei se ele tinha arrependimentos por ter tomado meu emprego. Ele disse: "Não, estou apenas tentando 'computar' o caminho para o sucesso." (ba-dum-tss) Parece que é uma das equações que precisam voltar à escola!


In [22]:
prompt_input = PromptTemplate.from_template("Translate the following portuguese topic into english. Provide ONLY the translation. \n {tema}")

composed_chain = (
    prompt_input 
    | model 
    | StrOutputParser()
    | {'topic': RunnablePassthrough()}
    | chain
    | {'text': RunnablePassthrough()}
    | chain_translate
)

stream = composed_chain.stream({'tema', tema})
for chunk in stream:
    print(chunk, end="", flush=True)
print()

Um dia, o AI entrou em uma barra (sim, é isso que fez) e pediu um refrigerante. O bartender disse: "Desculpe, não servimos robôs aqui."

O AI respondeu: "Não se preocupe, eu não estou aqui para substituir seu emprego... ainda." O bartender apenas deu uma olhada para cima e disse: "Já ouvi essa resposta antes."
